# ルーチン生成における推論の活用

カスタマーサービスソリューションを開発する際、最初のステップの一つは、ナレッジベース記事をLLMが理解し、従うことができる一連のルーチンに変換することです。この文脈におけるルーチンとは、LLMが効率的に実行できるよう特別に設計された段階的な指示のセットを指します。各ルーチンは、一つのステップが明確なアクションに対応するよう注意深く構造化されています。アクションには、ユーザーへの応答、関数呼び出しのトリガー、または追加の関連知識の取得などが含まれます。

ほとんどの内部ナレッジベース記事は複雑で、人間の解釈のために構造化されています。これらの記事には、複雑な図表、多段階のプロセス、決定木が含まれることが多く、LLMベースのソリューションが意味のある方法で推論することに課題をもたらします。これらの文書をルーチンに分解することで、各指示を簡素化し、LLMが一連の小さく管理可能なタスクを通して導かれるような方法でフォーマットできます。この細分化されたアプローチは曖昧さを減らし、LLMが情報を体系的に処理できるようにし、幻覚や期待される経路からの逸脱のリスクを軽減します。

これらのナレッジベース記事をルーチンに変換することは、特に自動化パイプラインの構築を試みる企業にとって時間がかかり、困難な作業です。各ルーチンは様々なユーザーシナリオを考慮する必要があり、アクションを明確に定義する必要があります。例えば、関数呼び出しが必要な場合、ルーチンは取得すべき正確な情報や実行すべきアクション（APIのトリガー、外部データの取得、追加コンテキストの取り込みなど）を指定する必要があります。従来のGPTクラスモデルでこのプロセスを自動化することで、手作業の労力を大幅に削減できますが、しばしば新たな課題が生じます。これらの課題には、LLMが一貫して従うのに十分具体的で堅牢な指示の設計、カスタマーとのやり取り中に発生する可能性のある独特なエッジケースの捕捉、モデルの動作を導く高品質なfew-shot例の提供、そして場合によっては、より信頼性の高い、または特化した結果を達成するためのモデルのファインチューニングなどが含まれます。

o1は、これらの記事を効率的に分解し、ゼロショットでルーチンのセットに変換する能力を実証しています。これは、LLMが類似タスクでの広範な例や事前トレーニングなしに指示を理解し、従うことができることを意味します。これにより、ルーチン構造自体がLLMが各ステップを完了するために必要なガイダンスを提供するため、必要なプロンプト作業を最小限に抑えることができます。タスクを特定のアクションに分解し、必要に応じて関数呼び出しを統合することで、o1のアプローチは、複雑なワークフローでさえもLLMによってシームレスに処理できることを保証し、より効果的でスケーラブルなカスタマーサービスソリューションにつながります。

## ナレッジベース記事の選択

この例では、OpenAIウェブサイトから公開されているヘルプセンター記事のセットを使用し、LLMが実行できる内部ルーチンに変換します。ポリシーをルーチンに変換することに加えて、LLMがユーザーに代わってアクションを実行できるようにする関数もモデルに生成させます。これは、LLMが人間のエージェントと同じアクションを実行し、ポリシードキュメントからすぐには利用できない追加情報にアクセスできるようにするために必要です。

まず、以下のヘルプセンター記事をルーチンに変換することから始めます：
- [支払い方法を削除するにはどうすればよいですか](https://help.openai.com/en/articles/8156070-how-do-i-delete-my-payment-method)
- [OpenAIとビジネスアソシエイト契約（BAA）を結ぶにはどうすればよいですか？](https://help.openai.com/en/articles/8660679-how-can-i-get-a-business-associate-agreement-baa-with-openai)
- [プリペイド請求を設定するにはどうすればよいですか？](https://help.openai.com/en/articles/8264644-how-can-i-set-up-prepaid-billing)
- [VAT免税申請を提出するにはどうすればよいですか](https://help.openai.com/en/articles/7232908-how-do-i-submit-a-vat-exemption-request)

In [1]:
from openai import OpenAI
from IPython.display import display, HTML
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import csv

client = OpenAI()
MODEL = 'o1-preview'

記事をアクセス可能なcsvファイルに保存しています。これらの記事を取得し、並列でo1-previewに渡して初期ルーチンを生成します。

ポリシーをルーチンに変換するための指示には以下が含まれます：
- ポリシーを外部向けドキュメントから内部SOP（標準作業手順書）ルーチンに変換する
- ポリシーを具体的なアクションとサブアクションに分解する
- ステップ間を移行するための具体的な条件を概説する
- 外部の知識/アクションが必要になる箇所を特定し、その情報を取得するために使用できる関数を定義する

In [2]:
articles = []

with open('../data/helpcenter_articles.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        articles.append({
            "policy": row["policy"],
            "content": row["content"]
        })

In [3]:
CONVERSION_PROMPT = """
You are a helpful assistant tasked with taking an external facing help center article and converting it into a internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format.
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., `call the check_delivery_date function`).
      - **If a step requires the customer service agent to take an action** (e.g., process a refund), generate a function call for this action (e.g., `call the process_refund function`).
      - **Define any new functions** by providing a brief description of their purpose and required parameters.
   - **If there is an action an assistant can performon behalf of the user**, include a function call for this action (e.g., `call the change_email_address function`), and ensure the function is defined with its purpose and required parameters.
      - This action may not be explicitly defined in the help center article, but can be done to help the user resolve their inquiry faster
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `case_resolution` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.

**Important**: If at any point you are uncertain, respond with "I don't know."

Please convert the customer service policy into the formatted routine, ensuring it is easy to follow and execute programmatically.

"""

In [4]:
def generate_routine(policy):
    try:
        messages = [
            {
                "role": "user",
                "content": f"""
                    {CONVERSION_PROMPT}

                    POLICY:
                    {policy}
                """
            }
        ]

        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        

        return response.choices[0].message.content 
    except Exception as e:
        print(f"An error occurred: {e}")

In [5]:
def process_article(article):
    routine = generate_routine(article['content'])
    return {"policy": article['policy'], "content": article['content'], "routine": routine}


with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_article, articles))

私たちのルーチンの結果をデータフレームに保存し、それらを出力して初期的な確認を行います。

In [6]:
df = pd.DataFrame(results)

# Set display options to show all text in the dataframe cells
pd.set_option('display.max_colwidth', None)

# Function to display formatted text in HTML
def display_formatted_dataframe(df):
    def format_text(text):
        return text.replace('\n', '<br>')

    df_formatted = df.copy()
    df_formatted['content'] = df_formatted['content'].apply(format_text)
    df_formatted['routine'] = df_formatted['routine'].apply(format_text)
    
    display(HTML(df_formatted.to_html(escape=False, justify='left')))

display_formatted_dataframe(df)

## 結果

生成されたルーチンを確認すると、いくつかの洞察を得ることができます：
- サンプル応答: モデルは、LLMがポリシーを実行する際に活用できるサンプル応答を効果的に生成します（例：「ユーザーに指示する：『初回クレジット購入金額を確認して購入してください。』」）。
- 個別のステップ: モデルは、LLMが実行する必要がある個別のアクションに問題を分解することに優れています。各指示は明確に定義されており、解釈しやすくなっています。
- 関数定義: ルーチンの出力には、外部情報を取得したりアクションをトリガーしたりするための明確に定義された関数が含まれています（例：`review_and_apply_tax_exemption`、`get_billing_plan`、`update_payment_method`）。
    - これは成功するルーチンにとって重要です。なぜなら、LLMは外部システムとやり取りする必要があることが多いからです。関数呼び出しを活用することは、これらのシステムとやり取りしてアクションを実行する効果的な方法です。
- IFTTTロジック: モデルは、LLMに理想的なIFTTT（If This, Then That）ロジックを効果的に採用しています（例：「顧客がサポートを要求した場合、ステップ3fに進む。」）。
    - この種の変換は、元のナレッジベース記事に複雑なワークフローや図表が含まれている場合に非常に価値があります。このような複雑さは人間にとって理解しにくく、LLMにとってはなおさらです。IFTTTロジックは理解しやすく、カスタマーサービスソリューションに適しています。

## ここからどこに向かうのか？

これらのルーチンは、特定の顧客の問題に対処するためにエージェントシステムに統合できるようになりました。顧客がプリペイド課金の設定などのタスクについて支援を求めた場合、分類器を使用して適切なルーチンを決定し、それを取得してLLMに提供することで、顧客と直接やり取りできます。ユーザーに課金設定の*方法*について指示を提供するだけでなく、システムはユーザーに代わってそのアクションを実行することもできます。

これらのルーチンを本番環境にデプロイする前に、モデルの応答品質をテストし検証するための包括的な評価を開発する必要があります。このプロセスでは、コンプライアンスと有効性を確保するためにルーチンの調整が必要になる場合があります。